![imagenes](logo.png)

# Tratamiento de valores faltantes

Cuando trabajamos con datos reales, tarde o temprano aparecen huecos: celdas en blanco, registros incompletos o valores dañados. En Python se representan con ``NaN``, en R con ``NA``, y en otros entornos con símbolos parecidos. Pero todos significan lo mismo: información que no está disponible.

Estos huecos pueden tener muchas causas. Tal vez un encuestado no respondió cierta pregunta, quizá un sensor falló en el momento de la medición, o simplemente un sistema de captura no registró el valor.

Imagina a un alumno que presenta un examen y deja varias preguntas sin contestar. El profesor puede tomar distintas decisiones: ignorar esas preguntas, calificarlas con cero o tratar de deducir cuál habría sido la respuesta correcta a partir del resto del examen. En un modelo de Machine Learning ocurre algo similar: debemos decidir qué hacer con la información que falta.

## Estrategias principales

Existen varias formas de tratar valores faltantes. Ninguna es universalmente mejor que otra: todo depende del contexto, de la cantidad de huecos y de la importancia de la columna en cuestión.

### Imputación simple

Otra estrategia es rellenar los huecos con valores representativos. Podemos usar la media, la mediana o la moda de la columna. También es posible asignar un número fijo, como cero o “desconocido”.

En Python, ``SimpleImputer`` de scikit-learn hace este trabajo de forma automática. Aquí el profesor del examen estaría “rellenando” las preguntas en blanco con la respuesta más común de los alumnos. No es perfecto, pero evita perder información.

### Imputación avanzada

Cuando el problema es más complejo, se pueden usar técnicas más sofisticadas. El **KNN Imputer** busca los registros más parecidos al que tiene el hueco y utiliza sus valores para completarlo. El **Iterative Imputer** construye modelos internos que predicen los valores faltantes a partir de las demás variables.

En nuestra analogía, Rocky entrena con sparrings que se parecen a su rival, para compensar los movimientos que no había practicado antes. Así llena sus vacíos con experiencias cercanas.

### Variables indicadoras

En ocasiones conviene crear una columna adicional que indique si un valor fue imputado o no. De esta forma el modelo puede aprender que la ausencia de datos en sí misma contiene información. En la práctica, esto es como que el profesor deje una marca en el examen para recordar que cierta respuesta fue rellenada artificialmente.

## La fuga de datos

Al igual que en la codificación de variables categóricas, aquí también debemos cuidar la fuga de datos. Nunca se deben calcular promedios o medianas usando todo el dataset antes de separar en entrenamiento y prueba.

Si lo hacemos, el modelo ya estaría “viendo” información del conjunto de prueba durante el entrenamiento. El paralelo en el examen sería que el niño recibe de antemano las respuestas del examen final disfrazadas en el cuestionario de práctica. El resultado será una calificación irrealmente alta.

Por eso, la regla de oro es:
1. Dividir primero en entrenamiento y prueba.
2. Ajustar (``fit``) el imputador solo con el entrenamiento.
3. Aplicar (``transform``) esa misma estrategia en entrenamiento y prueba.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

datos = pd.read_csv("datos_con_faltantes.csv")
datos.columns

In [ ]:
# Radiografía de faltantes (conteo y porcentaje)

faltantes = datos.isna().sum().sort_values(ascending=False)
porcentaje = (datos.isna().mean()*100).round(2).sort_values(ascending=False)
resumen_faltantes = pd.DataFrame({"faltantes": faltantes, "porcentaje_%": porcentaje})
print("\nResumen de faltantes por columna:\n")
print(resumen_faltantes, "\n")

In [ ]:
# Definir columnas cuantitativas

cols_quant_media   = ["ingreso"]           # imputar con MEDIA
cols_quant_mediana = ["gasto", "edad"]     # imputar con MEDIANA
cols_passthrough = ["completo"]

'''
Explicación del ColumnTransformer

transformers = [
  ("quant_media",
     SimpleImputer(strategy="mean", add_indicator=True),
     cols_quant_media),

  ("quant_mediana",
     SimpleImputer(strategy="median", add_indicator=True),
     cols_quant_mediana),

  ("passthrough",
     "passthrough",
     [todas las demás columnas])
]

- MEDIA para columnas donde la distribución es aproximadamente simétrica.
- MEDIANA para columnas sesgadas o con outliers.
- add_indicator=True añade banderas de imputación por columna numérica.
- remainder="passthrough" conserva todo lo no listado (categóricas, 'completo', etc.) tal cual.
'''



preprocesador = ColumnTransformer(
    transformers=[
        ("quant_media",
         SimpleImputer(strategy="mean", add_indicator=False),
         cols_quant_media),

        ("quant_mediana",
         SimpleImputer(strategy="median", add_indicator=False),
         cols_quant_mediana),

        ("passthrough",
         "passthrough",
         cols_passthrough),
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

# 
# Ajustar y transformar
# preprocesador.fit(datos)

X_imp    = preprocesador.transform(datos)
cols_out = preprocesador.get_feature_names_out()

df_imputado = pd.DataFrame(X_imp, columns=cols_out, index=datos.index)
df_imputado


In [ ]:
faltantes = df_imputado.isna().sum().sort_values(ascending=False)
porcentaje = (df_imputado.isna().mean()*100).round(2).sort_values(ascending=False)
resumen_faltantes = pd.DataFrame({"faltantes": faltantes, "porcentaje_%": porcentaje})
print("\nResumen de faltantes por columna:\n")
print(resumen_faltantes, "\n")